## WordPiece Tokenizer Basic

### Описание задачи

WordPiece — это алгоритм субсловной токенизации, похожий на BPE, но с другим принципом объединения.
WordPiece использует языковую модель для оценки возможных слияний, выбирая такие пары, которые максимизируют правдоподобие обучающих данных.

WordPiece используется в BERT и многих моделях Google. Он создает субсловные токены, которые начинаются с `##`, чтобы показать, что это не начало слова.

### Зачем нужен WordPiece

WordPiece важен, потому что:

- **BERT**: используется в моделях BERT
- **Google models**: стандарт для моделей обработки текста Google
- **Subword units**: эффективно обрабатывает неизвестные слова
- **Production**: применяется в продакшн‑системах
- **OOV handling**: лучше обрабатывает слова вне словаря, чем токенизация по словам

### Математическое описание (в общих чертах)

Алгоритм WordPiece:

1. **Инициализация**: начать с токенов на уровне символов.
2. **Оценка слияний**: оценивать возможные слияния с помощью правдоподобия языковой модели.
3. **Слияние**: сливать пару, которая дает наибольший прирост правдоподобия.
4. **Повторение**: повторять, пока не будет достигнут целевой размер словаря.

Ключевое отличие от BPE:

- BPE: объединяет наиболее частую пару.
- WordPiece: объединяет пару, дающую максимальный рост правдоподобия.

---

## Задание

Реализовать функцию `wordpiece_tokenize(text, vocab)`.

**Входные данные:**

- `text`: входная строка текста;
- `vocab`: множество (`set`) или словарь (`dict`) допустимых WordPiece‑токенов.

**Выходные данные:**

- список строк‑токенов.

### Требуемый алгоритм (упрощённая версия)

Реализовать жадную токенизацию по правилу «самое длинное совпадение»:

1. На каждом шаге брать **самое длинное возможное совпадение** с начала текущей подстроки.
2. Если совпадение найдено в словаре `vocab`, добавить этот токен в результат и продолжить с остатком строки.
3. Если совпадения нет, добавить токен для неизвестного слова (например, `"<UNK>"`).
4. Вернуть список токенов.

Это упрощенный жадный вариант; полный WordPiece использует более сложное сопоставление.

### Ограничения и требования

- Использовать чистый Python.
- Реализовать жадную токенизацию с выбором **максимально длинного совпадения**.
- Корректно проверять токены на наличие в словаре `vocab`.
- Обязательно обрабатывать неизвестные токены (`<UNK>` или аналог).
- Вернуть список строк‑токенов.

---

## Пример

```python
text = "unhappiness"
vocab = {
    "un",
    "##happy",
    "##ness",
    "un##happy",
    "un##happy##ness",
    "<UNK>"
}

tokens = wordpiece_tokenize(text, vocab)
# Возможные корректные ответы:
# ["un##happy##ness"]
# или ["un", "##happy", "##ness"]


In [ ]:
def wordpiece_tokenize(text, vocab, unk_token="<UNK>"):
    if isinstance(vocab, dict): vocab = set(vocab.keys())
    else: vocab = set(vocab)
    output = []

    for word in text.strip().split():
        start, failed = 0, False
        word_tokens = []

        while start < len(word):
            best = None
            for end in range(len(word), start, -1):
                piece = word[start:end]
                token = piece if start == 0 else "##" + piece
                if token in vocab:
                    best = token
                    break

            if best is None:
                failed = True
                break
            word_tokens.append(best)
            start += len(best) if not best.startswith("##") else len(best) - 2
        if failed: output.append(unk_token)
        else: output.extend(word_tokens)
    return output